In [1]:
import dlib
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow import keras

In [2]:
model = load_model('my_model')

2022-10-04 09:57:56.362279: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-04 09:57:56.390431: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-04 09:57:56.390615: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-04 09:57:56.391226: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
predictor = dlib.shape_predictor('shape_68.dat')
detector = dlib.get_frontal_face_detector()

In [4]:
#model.summary()

In [5]:
def class_names(number):
    if number == 0:
        return "Center"
    elif number == 1:
        return "Up"
    elif number == 2:
        return "Down"
    elif number == 3:
        return "Right"
    elif number == 4:
        return "Left"
    elif number == 5:
        return "Closed"
    else:
        return "Unknown"


In [6]:

def generate_predictions(camera):
    ret, frame = camera.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)
    for (i, rect) in enumerate(rects):
        shape = predictor(gray, rect)
    x_eye_points = [shape.part(x).x for x in range(37, 42)]
    y_eye_points = [shape.part(x).y for x in range(37, 42)]
    padding = 8
    maxx = max(x_eye_points)
    minx = min(x_eye_points)
    maxy = max(y_eye_points)
    miny = min(y_eye_points)
    crop_image = frame[miny-padding:maxy +
                        padding, minx-padding:maxx+padding]
    resize = cv2.resize(crop_image, (32, 32))
    #resize = cv2.flip(resize,1)
    cv2.imshow("Output", resize)
    resize = np.expand_dims(resize, axis=0)
    predicted = model.predict(resize)
    predicted_class = np.argmax(predicted)
    cv2.waitKey(1)
    return (class_names(predicted_class))

In [7]:
cap = cv2.VideoCapture(0)
while True:
    print(generate_predictions(cap))

NameError: free variable 'shape' referenced before assignment in enclosing scope